# 协议与接口

1. 我们把协议定义为非正式的接口，是让 Python 这种动态类型语言实现多态的方式。
2. 接口：类实现或继承的公开属性（方法或数据属性），包括特殊方法，如 `__getitem__` 或 `__add__`。补充定义：对象公开方法的子集，让对象在系统中扮演特定的角色。
3. 协议是接口，但不是正式的（只由文档和约定定义），因此协议不能像正式接口那样施加限制.
4. 一个类可能只实现部分接口，这是允许的。有时，某些 API 只要求“文件类对象”返回字节序列的 .read() 方法。在特定的上下文中可能需要其他文件操作方法，也可能不需要。

# 猴子补丁

在运行时修改类或模块，而不改动源码。 猴子补丁很强大，但是打补丁的代码与要打补丁的程序耦合十分紧密，而且往往要处理隐藏和没有文档的部分。

# 继承抽象基类 实现一个洗牌功能

In [1]:
import collections

我们明确把 FrenchDeck 声明为 collections.MutableSequence 这个抽象基类的子类。

In [2]:
Card = collections.namedtuple('Card', ['rank', 'suit'])

In [3]:
class FrenchDeck(collections.MutableSequence):
    # 2.3.4...10.J.Q.K.A
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    # 四种花色
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, pos):
        return self._cards[pos]
    
    def __setitem__(self, pos, value):
        self._cards[pos] = value
        
    def __delitem__(self, pos): # (1)
        del self._cards[pos]
        
    def insert(self, pos, value): # (2)
        self._cards.insert(pos, value)

/Users/feipeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


(1). 继承 MutableSequence 的类必须实现 `__delitem__` 方法，这是 MutableSequence 类的一个抽象方法。

(2). 此外，还要实现 `insert` 方法，这是 MutableSequence 类的第三个抽象方法。

# 定义一个抽象基类

除非高级python开发者，否则不要自己定义抽象基类！！！

但作为说明，我们斗胆定义一个抽象基类。

声明抽象基类最简单的方式是继承 `abc.ABC` 或其他抽象基类。

In [4]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""
        
    @abc.abstractmethod
    def pick(self):
        """随机删除元素，然后将其返回
        如果实例为空，抛出LookupError
        """
    
    def loaded(self):
        """如果至少有一个元素，返回true，否则返回false"""
        return bool(self.inspect())
    
    def inspect(self):
        """返回一个有序元祖，由当前元素构成"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

Tombola 是抽象基类，有两个抽象方法和两个具体方法
1. 自己定义的抽象基类要继承 `abc.ABC`。
2. 抽象方法使用 `@abstractmethod` 装饰器标记，而且定义体中通常只有文档字符串。
3. 抽象基类可以包含具体方法。
4. 抽象基类中的具体方法只能依赖抽象基类定义的接口（即只能使用抽象基类中的其他具体方法、抽象方法或特性）。

其实，抽象方法可以有实现代码。即便实现了，子类也必须覆盖抽象方法，但是在子类中可以使用 `super()` 函数调用抽象方法，为它添加功能，而不是从头开始实现。

下面看看如果不按照抽象基类的规则使用，会出现什么情况。

In [5]:
class Fake(Tombola):
    
    def pick(self):
        return 12
    
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

因为它没有实现 `load` 方法，所以报错。

除了 `@abstractmethod` 之外， `abc` 模块还定义了 `@abstractclassmethod`、 `@abstractstaticmethod` 和 `@abstractproperty` 三个装饰器。然而，后三个装饰器从 Python 3.3 起废弃了，因为装饰器可以在 `@abstractmethod`上堆叠，那三个就显得多余了。例如，声明抽象类方法的推荐方式是：

In [6]:
class MyABC(abc.ABC):
    
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls):
        pass

在函数上堆叠装饰器的顺序通常很重要， `@abstractmethod` 的文档就特别指出：与其他方法描述符一起使用时， `abstractmethod()` 应该放在最里层。也就是说，在 `@abstractmethod` 和 `def` 语句之间不能有其他装饰器。

# 定义抽象基类的子类

In [7]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        return self.pick()

In [8]:
bingocage = BingoCage(range(3))

In [9]:
bingocage()

1

BingoCage 从 Tombola 中继承了耗时的 loaded 方法和笨拙的 inspect 方法。这两个方法都可以覆盖，变成下例中速度更快的一行代码。这里想表达的观点是：我们可以偷懒，直接从抽象基类中继承不是那么理想的具体方法。从 Tombola 中继承的方法没有 BingoCage 自己定义的那么快，不过只 Tombola 的子类正确实现 pick 和 load 方法，就能提供正确的结果。

In [10]:
import random

class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable) 
    
    def load(self, iterable):
        self._balls.extend(iterable)
    
    def pick(self):
        try:
            position = random.randrange(len(self._balls)) 
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(position) 
    
    def loaded(self): 
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

# 虚拟子类

注册虚拟子类的方式是在抽象基类上调用 `register` 方法。这样做时，我们要保证注册的类忠实地实现了抽象基类定义的接口，而 Python 会相信我们，从而不做检查。如果说谎了，那么常规的运行时异常会被捕获。

这么做之后，注册的类会变成抽象基类的虚拟子类，而且 `issubclass` 和 `isinstance` 等函数都能识别，但是注册的类不会从抽象基类中继承任何方法或属性。

虚拟子类不会继承注册的抽象基类，而且任何时候都不会检查它是否符合抽象基类的接口，即便在实例化时也不会检查。为了避免运行时错误，虚拟子类要实现所需的全部方法。

`register` 方法通常作为普通的函数调用，不过也可以作为装饰器使用。

In [11]:
@Tombola.register  # 注册为 Tombola 的虚拟子类。
class TomboList(list):  # 扩展 list

    def pick(self):
        if self:  # 从 list 中继承 __bool__ 方法，列表不为空时返回 True。
            position = randrange(len(self))
            return self.pop(position)  # 调用继承自 list 的 self.pop 方法，传入一个随机的元素索引。
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  # 与 list.extend 一样

    def loaded(self):
        return bool(self)  # 委托 bool 函数。

    def inspect(self):
        return tuple(sorted(self))

# 子类测试

测试子类时用到 2 个类属性， `__subclasses__()` 和 `_abc_registry` .

`__subclasses__`  
这个方法返回类的直接子类列表，不含虚拟子类。

`_abc_registry`  
只有抽象基类有这个数据属性，其值是一个 WeakSet 对象，即抽象类注册的虚拟子类的弱引用。

`__subclasshook__`特殊方法  
可以完全使用不相关的类，只要实现特定的方法即可, 当然，只有提供 `__subclasshook__` 方法的 抽象类 才能这么做。


In [12]:
Tombola.__subclasses__()

[__main__.Fake, __main__.BingoCage, __main__.LotteryBlower]

In [16]:
from collections import abc

class Struggle:
    def __len__(self):
        return 10

In [18]:
isinstance(Struggle(), abc.Sized)

True

In [19]:
issubclass(Struggle, abc.Sized)

True

经 issubclass 函数确认（ isinstance 函数也会得出相同的结论）， Struggle 是 abc.Sized 的子类，这是因为 abc.Sized 实现了一个特殊的类方法，名为 `__subclasshook__`。

```python
class Sized(metaclass=ABCMeta):

    __slots__ = ()   # 防止被动态赋予属性？

    @abstractmethod
    def __len__(self):
        return 0

    @classmethod
    def __subclasshook__(cls, C):  # C 是否为 cls 的子类.
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented  # 返回 让子类检查。
```

在自己定义的抽象基类中要不要实现 `__subclasshook__` 方法呢？可能不需要。在你我自己编写的抽象基类中实现 `__subclasshook__` 方法，可靠性很低。我可不相信随便一个实现或继承了 load、 pick、 inspect 和 loaded 的类的行为一定像 Tombola。